<a href="https://colab.research.google.com/github/mushrafi88/asr_bangla/blob/main/gpt_2_asr_bn_corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Install libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install huggingface
!pip install datasets tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 56.4 MB/s 
     |████████████████████████████████| 213 kB 57.2 MB/s 
     |████████████████████████████████| 182 kB 68.6 MB/s 
     |████████████████████████████████| 127 kB 59.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 
     |████████████████████████████████| 7.

In [3]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm import tqdm

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
pd.set_option('display.max_colwidth', None)

In [5]:
from transformers import (TrainingArguments,Trainer,AutoTokenizer,AutoModelForSeq2SeqLM
  )

from torch.utils.data import Dataset, DataLoader

In [62]:
df = pd.read_csv('/content/drive/MyDrive/bert_asr/bert_corrector/data/train/raw.csv')
df = df[['wav2vec2','sentence']]
df = df.dropna(how='any')

In [63]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt2-bengali")

model = AutoModelForCausalLM.from_pretrained("flax-community/gpt2-bengali")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--flax-community--gpt2-bengali/snapshots/cb8fff6e5e2c459c057ce2d1a8e14fd79bb0f0a1/config.json
Model config GPT2Config {
  "_name_or_path": "flax-community/gpt2-bengali",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.0,
  "bos_token_id": 50256,
  "embd_pdrop": 0.0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.0,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type"

In [64]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [65]:
df = df.rename(columns={"wav2vec2": "input", "sentence": "output"})

In [66]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((155459, 2), (17274, 2))

In [67]:
test_df.head()

,input,output
37967,সাব্নডের কর্মকান্ডেহতার,শাবনূরের কর্মকাণ্ডে হতাশ
15132,তার আয়ত্বাধীন,তাঁর আয়ত্তাধীন
95909,তামিল নেতৃত্ব এই পরিকল্পনা তীব্র বিরোধিতা করে বুজতে ফেরেছিলেন যে তারা সংসদে সংখ্যালোগু হয়ে পড়বেন,তামিল নেতৃত্ব এই পরিকল্পনার তীব্র বিরোধিতা করে বুঝতে পেরেছিলেন যে তারা সংসদে সংখ্যালঘু হয়ে পড়বে
148798,তিনি পাকিস্তানের সংবিধানে রষ্টম সংসদনী বাতিলের প্রতিশ্রুতি দিয়েছিলেন এবং প্রধানমন্ত্রী বিন্তর ভূট্তর পক্ষে সমর্থন প্রকাশ করেছিলেন,তিনি পাকিস্তানের সংবিধানের অষ্টম সংশোধনী বাতিলের প্রতিশ্রুতি দিয়েছিলেন এবং প্রধানমন্ত্রী বেনজির ভুট্টোর পক্ষে সমর্থন প্রকাশ করেছিলেন
71520,স্বাভাবিকে তুলনায় দ্ীগুনের বেশি,স্বাভাবিকের তুলনায় দ্বিগুণের বেশি


In [68]:
tokenizer(df.input[30542])

{'input_ids': [3749, 266, 706, 625, 258, 280], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [69]:
#test_df['input_token_len'].describe()

### We will use a token length of 64 since it will cover the vast majority of examples

In [70]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [71]:
train_dataset.save_to_disk('/content/train')
test_dataset.save_to_disk('/content/test')

Saving the dataset (0/1 shards):   0%|          | 0/155459 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/17274 [00:00<?, ? examples/s]

In [72]:
train_dataset[1]

{'input': 'লেখা গনিত ব', 'output': 'লেখা গণিত বই', '__index_level_0__': 44195}

### Load the Dataset

In [73]:
max_len = 64 

In [74]:
train_dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 155459
})

In [75]:
tokenizer(train_dataset['output'][0])

{'input_ids': [280, 262, 404, 258, 555, 286, 550, 258, 260, 759, 353, 258, 299, 264, 1194, 258, 282, 262, 260, 324, 262, 276, 386, 276, 264, 272, 262, 324, 266, 260, 264, 272, 258, 295, 317, 293, 266, 333, 258, 308, 435, 266, 293, 266, 323, 258, 260, 293, 266, 333, 258, 308, 412, 350, 320, 262, 404, 258, 555, 286, 293, 266, 337, 258, 3485, 258, 324, 258, 280, 262, 435, 266, 13390, 258, 324, 266, 274, 328, 2051, 266, 276], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [80]:
tokenizer.add_special_tokens({'pad_token': '<|padding|>'})

Assigning <|padding|> to the pad_token key of the tokenizer


1

In [81]:
def prepare_data(example):
  example["input_ids"] = tokenizer(example['input'], padding=True, return_attention_mask = True).input_ids
  example["attention_mask"] = tokenizer(example['input'],padding=True, return_attention_mask = True).attention_mask
  example["labels"] = tokenizer(example["output"],padding=True).input_ids[0]
  return example

In [82]:
train_dataset[0]

{'input': 'মেঘালয় সরকার একজন রাজ্যপালের নেতৃত্বে নির্বাহী বভাগ একটি বিচার বিভাগ এবং মেঘালয় বিধানসভা নামে একি আইনসভা নিয়ে গঠিত',
 'output': 'মেঘালয় সরকার একজন রাজ্যপালের নেতৃত্বে নির্বাহী বিভাগ একটি বিচার বিভাগ এবং মেঘালয় বিধানসভা নামে একটি আইনসভা নিয়ে গঠিত',
 '__index_level_0__': 90839}

In [83]:
tokenizer

PreTrainedTokenizerFast(name_or_path='flax-community/gpt2-bengali', vocab_size=50256, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|padding|>'})

In [84]:
train_data = train_dataset.map(prepare_data,remove_columns=train_dataset.column_names)

  0%|          | 0/155459 [00:00<?, ?ex/s]

In [85]:
test_data = test_dataset.map(prepare_data)

  0%|          | 0/17274 [00:00<?, ?ex/s]

In [86]:
tokenizer.decode(train_data[0]['labels'])

'মেঘালয় সরকার একজন রাজ্যপালের নেতৃত্বে নির্বাহী বিভাগ একটি বিচার বিভাগ এবং মেঘালয় বিধানসভা নামে একটি আইনসভা নিয়ে গঠিত'

### Train Model

In [87]:
%mkdir -p /content/drive/MyDrive/GPT_model_raw_sentence/weights

In [88]:
from huggingface_hub import notebook_login

notebook_login()

In [89]:
# defining training related arguments
batch_size = 8
args = TrainingArguments(output_dir="/content/drive/MyDrive/GPT_model_raw_sentence/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=1e-5,
                        num_train_epochs=2,
                        weight_decay=0.01,
                        save_total_limit=2,
                        #fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        #push_to_hub=True,
                       )
                                         

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [90]:
# defining trainer using 🤗
trainer = Trainer(model=model, 
                args=args, 
                train_dataset= train_data,
                eval_dataset=test_data,
                tokenizer=tokenizer)

In [91]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [93]:
trainer.train()

***** Running training *****
  Num examples = 155459
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 6
  Total optimization steps = 6476
  Number of trainable parameters = 124439808


ValueError: ignored

In [ ]:
trainer.save_model('t5_asr_bn_corrector')

Saving model checkpoint to t5_asr_bn_corrector
Configuration saved in t5_asr_bn_corrector/config.json
Model weights saved in t5_asr_bn_corrector/pytorch_model.bin
tokenizer config file saved in t5_asr_bn_corrector/tokenizer_config.json
Special tokens file saved in t5_asr_bn_corrector/special_tokens_map.json
Saving model checkpoint to mushrafi88/T5-asr-corrector
Configuration saved in mushrafi88/T5-asr-corrector/config.json
Model weights saved in mushrafi88/T5-asr-corrector/pytorch_model.bin
tokenizer config file saved in mushrafi88/T5-asr-corrector/tokenizer_config.json
Special tokens file saved in mushrafi88/T5-asr-corrector/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/944M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mushrafi88/T5-asr-corrector
   c9b56ef..c92e63f  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/mushrafi88/T5-asr-corrector
   c9b56ef..c92e63f  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/mushrafi88/T5-asr-corrector
   c92e63f..af3aaae  main -> main

   c92e63f..af3aaae  main -> main



In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mushrafi88/T5-asr-corrector")

model = AutoModelForSeq2SeqLM.from_pretrained("mushrafi88/T5-asr-corrector")

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file mushrafi88/T5-asr-corrector/config.json
Model config MT5Config {
  "_name_or_path": "mushrafi88/T5-asr-corrector",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping"

## Testing

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/bert_asr/bert_corrector/data/test/symspell_nov.csv')

In [ ]:
df_test = df_test[['sentence','wav2vec2','4gram']]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(model_name).to('cuda')

loading configuration file mushrafi88/T5-asr-corrector/config.json
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Model config T5Config {
  "_name_or_path": "flax-community/bengali-t5-base",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
  

In [ ]:
from transformers import pipeline

generator = pipeline(model=model_name, tokenizer=tokenizer)

loading configuration file mushrafi88/T5-asr-corrector/config.json
Model config MT5Config {
  "_name_or_path": "mushrafi88/T5-asr-corrector",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": 

In [ ]:
df_test.wav2vec2[1500]

'তাই মসার আবাস্থ উদ্ধ্বশ করে মসাব বং বিস্তাবক্রতিরোধ করতে হবে'

In [ ]:
generator(df_test.wav2vec2[1500])

[{'generated_text': 'তাই মসরাশবসায় মসরাশবসা'}]